In [1]:
from time import time
import torch
import numpy as np
from torch.utils.data import DataLoader
from data import TargetData
from models import NCF
from evaluators import eval_model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

# -----------------------------------------------------------------

device(type='cuda')

In [2]:
print("Processing data...")
data = TargetData()
print("Done")

# -----------------------------------------------------------------

Processing data...
Done


In [3]:
num_epochs = 5
learning_rate = .1

emb_size = 128
hidden_layers = np.array([emb_size, 64, 32, 16])
output_size = 1
random_samples = 100
num_negatives = 4
top_k = 10
batch_size = 256

# -----------------------------------------------------------------

In [4]:
data.test.head()

# -----------------------------------------------------------------

,uid,mid,rating
25,0,25,1
66,1,66,1
232,2,207,1
235,3,208,1
258,4,222,1


In [5]:
def train_ncf(model):
    # data.get_train_instances(seed=e)
    optimizer = torch.optim.AdamW(model.parameters(),  lr=learning_rate, weight_decay=1e-6)
    # optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-6)
    dataloader = DataLoader(data, batch_size=batch_size,
                        shuffle=True, num_workers=0)
    t1 = time()


    it_per_epoch = len(data) / batch_size
    for i in range(num_epochs):
        ncf.train()
        print("Starting epoch ", i + 1)
        j = 0
        # loss = 0
        for batch in dataloader:
            u, m, r = batch
            # move tensors to cuda
            u = u.to(device)
            m = m.to(device)
            r = r.to(device)

            y_hat = model(u.squeeze(1), m.squeeze(1))
            loss = torch.nn.BCELoss()  # (weight=w, reduction="mean")

            loss = loss(y_hat, r.float())
            optimizer.zero_grad()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # if j % int(1 + it_per_epoch / 10) == 0:
            #     print("Progress: ", round(100 * j / it_per_epoch), "%")
            j+=1
        # Epoch metrics
        t2 = time()
        # print("Epoch time:", round(t2 - t1), "seconds")
        # print("Loss:", loss.cpu().detach().numpy().round(3) / i)

        # print("Evaluating model...")
        ncf.eval()
        hr, ndcg = eval_model(model, data)
        print(f"HR@{top_k}: {round(hr, 2)}")
        # print(f"NDCG@{top_k}: {round(ndcg, 2)}")
    print("Done")

# def fed_fit(model_central, data, C, batch_size, epochs, lr, eta, verbose=True):
#
#     # Sample the participants for the round of training
#     num_participants = int(data.num_users * C)
#     participants = random.sample(range(data.num_users), num_participants)
#
#     # model_difference holds the total change of the global model after the round
#     model_difference = copy.deepcopy(model_central)
#     utils.zero_model_parameters(model_difference)
#
#     it = 0
#
#     t1 = time()
#
#     # Start training loop
#     for user in participants:
#
#         it += 1
#         if it % int(num_participants / 10) == 0 and verbose:
#             print("Progress:", round(100 * it / num_participants), "%")
#
#         # The current user takes a copy of the global model
#         model_client = copy.deepcopy(model_central)
#
#         # Defining optimizers
#         optimizer = torch.optim.SGD(model_client.parameters(), lr=lr)  # MLP optimizer
#         optimizer_u = torch.optim.SGD(model_client.user_embedding.parameters(), lr=lr / C * eta - lr)  # User optimizer
#         optimizer_i = torch.optim.SGD(model_client.item_embedding.parameters(),
#                                       lr=lr * data.num_items * eta - lr)  # Item optimizer
#
#         # Prepares data for the current user
#         # data.set_current_user(user)
#         # data.generate_negatives()
#
#         dataloader = DataLoader(data, batch_size=batch_size,
#                                 shuffle=True, num_workers=0)
#
#         # Trains on the users data
#         for e in range(epochs):
#             for batch in dataloader:
#                 # Load tensors of users, movies, outputs and loss weights
#                 u, m, y = batch
#                 # move tensors to cuda
#                 u = u.to(device)
#                 m = m.to(device)
#                 y = y.to(device)
#                 # w = w.to(device)
#
#                 # make predictions
#                 p_pred = model_client(u, m)
#
#                 # Calculate mean loss
#                 loss_fn = torch.nn.BCELoss()  # weight=w, reduction="mean")
#                 loss = loss_fn(p_pred, y)
#
#                 # Backpropagate the output and update model parameters
#                 optimizer.zero_grad()
#                 optimizer_u.zero_grad()
#                 optimizer_i.zero_grad()
#
#                 loss.backward()
#                 optimizer.step()
#                 optimizer_u.step()
#                 optimizer_i.step()
#
#         # Calculate the user's change of the model and add it to the total change
#         utils.sub_model_parameters(model_central, model_client)
#         utils.add_model_parameters(model_client, model_difference)
#
#     # Take the average of the MLP and item vectors
#     utils.divide_model_parameters(model_difference, num_participants)
#
#     # Update the global model by adding the total change
#     utils.add_model_parameters(model_difference, model_central)
#     t2 = time.time()
#     print("Time of round:", round(t2 - t1), "seconds")

# def train_fed():
#     for t in range(T):  # for each round
#         print("Starting round", t + 1)
#         # train one round
#         fed_fit(model_central, data, C=C, batch_size=batch_size, epochs=E, lr=learning_rate, eta=eta, verbose=True)
#         print("Evaluating model...")
#         # HR, NDCG = evaluate_model(model_central, data, validation=False)
#         # hr, ndcg = evaluate_model(
#         #     NCF,
#         #     data.test.values,
#         #     25,
#         #     random_samples,
#         #     data.num_movies
#         # )
#         print("HR@10:", hr)
#         print("NDCG@10", ndcg)

# -----------------------------------------------------------------

In [6]:
ncf = NCF(data.num_users, data.num_movies, emb_size, hidden_layers, output_size).to(device)

train_ncf(ncf)

Starting epoch  1
HR@10: 0.99
Starting epoch  2
HR@10: 1.0
Starting epoch  3
HR@10: 1.0
Starting epoch  4
HR@10: 1.0
Starting epoch  5
HR@10: 1.0
Done


In [8]:
torch.save(ncf.state_dict(), "saved_models/preTrained_NCF")
